In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import random

In [49]:
'''
Do stratified random selection of year indices for increasing increments of sample sizes 
while retaining indices of previous sampling. 
'''

# LOAD EXAMPLE FILE
example = xr.open_dataset('/nesi/project/niwa00018/ML_downscaling_CCAM/multi-variate-gan/inputs/predictor_fields/predictor_fields_hist_ssp370_merged_updated.nc')

years = np.unique(example.time.dt.year)

def stratified_sampling(array,sampling):

    selected_years = []
    for ar in np.array_split(np.array(array),sampling):

        selected_years.append(random.sample(list(ar),1)[0])
        
    return(selected_years)


# DO SELECTION
increments = [5,10,20,30,40,50,60,70,80,90,100,110,120,130,140]

years_left = years
samplings = {}

previous_inc = 0
previous_years = []
for inc in increments:
    sample_n = inc - previous_inc
    
    selected_years = stratified_sampling(years_left,sample_n)
    samplings[inc] = previous_years + selected_years

    
    years_left = [val for val in years if val not in samplings[inc]]
    
    previous_inc = inc
    previous_years = samplings[inc]

# SAVE YEAR SAMPLINGS

years_df = pd.DataFrame({sample_n:pd.Series(years) for sample_n,years in samplings.items()})


'\nfor sample_n in years_df:\n    print(sample_n)\n\n    selected_years = years_df[sample_n].values\n    selected_years = selected_years[~np.isnan(selected_years)]\n    selected_years = [int(v) for v in selected_years]\n\n    unselected_years = [val for val in years if val not in selected_years]\n\n    print(len(selected_years))\n    print(len(unselected_years))\n    \n    print()\n'

In [60]:
'''
DO NOT OVERRIDE TEMPORAL SAMPLING FILE
'''

#years_df.to_csv('temporal_samplings_2.csv',index=False)
